# SCRAPPING


In [ ]:
!pip install google_play_scraper
!pip install textblob
!pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.4 MB/s eta 0:00:00


In [ ]:
from google_play_scraper import app
import pandas as pd
import numpy as np
import sklearn
import requests
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import seaborn as sns
import textblob
#from wordcloud import WordCloud
from pathlib import Path
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score

import pickle
import re
import time
import datetime                              # access to %%time, for timing individual notebook cells
import os
from PIL import Image
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

%matplotlib inline
%config InlineBackend.figure_format='retina'

# Import seaborn styles explicitly
import seaborn as sns
# Apply the seaborn style before creating plots
sns.set_style("whitegrid")  # This line sets the Seaborn style

plt.rcParams["figure.figsize"] = (15,10)

SCRAPPING

In [ ]:
#Android App Mobile Maxim  from Google Play at
#https://play.google.com/store/apps/details?id=app.bpjs.mobile&hl=id
# https://play.google.com/store/apps/details?id=com.taxsee.taxsee
#The apps ID found in the link after id=com.taxsee.taxsee
#The apps name on Google Play titled: Mobile Maxim
#Dated 3 March 2025 and N of reviews 787K

from google_play_scraper import app, Sort, reviews_all

mx_reviews = reviews_all(
    'com.taxsee.taxsee',
    sleep_milliseconds=0, # defaults to 0
    lang='id', # defaults to 'en'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

In [ ]:
#Save Mobile Maxim reviews into dataframe
df_mxrev = pd.DataFrame(np.array(mx_reviews),columns=['content'])
df_mxrev = df_mxrev.join(pd.DataFrame(df_mxrev.pop('content').tolist()))
df_mxrev.to_csv('MaximRaw.csv', index=False)  #Save Into CSV

DATA FRAME WITH COLLECTED COLUMNS

In [ ]:
#See Data Frame info
df_mxrev
df_mxrev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477188 entries, 0 to 477187
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              477188 non-null  object        
 1   userName              477188 non-null  object        
 2   userImage             477188 non-null  object        
 3   content               477187 non-null  object        
 4   score                 477188 non-null  int64         
 5   thumbsUpCount         477188 non-null  int64         
 6   reviewCreatedVersion  427539 non-null  object        
 7   at                    477188 non-null  datetime64[ns]
 8   replyContent          241661 non-null  object        
 9   repliedAt             241661 non-null  datetime64[ns]
 10  appVersion            427539 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 40.0+ MB


In [ ]:
#We do not need data for column reviewId, userName, userImage so
#we will show only these column
df_mxan = df_mxrev.loc[:,["content","score","thumbsUpCount", "reviewCreatedVersion", "at", "replyContent", "repliedAt"]]

In [ ]:
#Save Dataframe into CSV
df_mxan.to_csv("MaximReview.csv", index=False)

# PRE-PROCESSING


Membuat DF Baru berisi 50 baris utama

In [ ]:
df_mxan_50 = df_mxan.sample(n=50, random_state=1)

## TOKENIZATION

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd

# Download punkt if not already present
nltk.download('punkt')
nltk.download('punkt_tab') # Download the punkt_tab resource

def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

# Apply tokenization to the 'content' column
df_mxan_50['tokenized_content'] = df_mxan_50['content'].astype(str).apply(tokenize_text)

# Display the DataFrame with the new 'tokenized_content' column
df_mxan_token = df_mxan_50[['content', 'tokenized_content']]

df_mxan_token


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,content,tokenized_content
325768,Murah ya.. bisa buat langganan nih 👌 semoga te...,"[Murah, ya, .., bisa, buat, langganan, nih, 👌,..."
373079,ZTE sdr deRr Cut PT,"[ZTE, sdr, deRr, Cut, PT]"
13536,oklh,[oklh]
31892,sangat baik,"[sangat, baik]"
442320,the best,"[the, best]"
431309,oke,[oke]
86580,sangat baik,"[sangat, baik]"
9542,mantap,[mantap]
328152,anu enak,"[anu, enak]"
341287,driver yang baik,"[driver, yang, baik]"


In [ ]:
#Save Dataframe into CSV
df_mxan_token.to_csv("MaximReview_Token.csv", index=False)

## LOWERING CASE

In [ ]:
def lower_tokenized(tokens):
    return [token.lower() for token in tokens]

df_mxan_token['lower_tokenized_content'] = df_mxan_token['tokenized_content'].apply(lower_tokenized)
df_mxan_lower = df_mxan_token[['tokenized_content', 'lower_tokenized_content']]
df_mxan_lower



/tmp/ipython-input-2513985902.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mxan_token['lower_tokenized_content'] = df_mxan_token['tokenized_content'].apply(lower_tokenized)


,tokenized_content,lower_tokenized_content
325768,"[Murah, ya, .., bisa, buat, langganan, nih, 👌,...","[murah, ya, .., bisa, buat, langganan, nih, 👌,..."
373079,"[ZTE, sdr, deRr, Cut, PT]","[zte, sdr, derr, cut, pt]"
13536,[oklh],[oklh]
31892,"[sangat, baik]","[sangat, baik]"
442320,"[the, best]","[the, best]"
431309,[oke],[oke]
86580,"[sangat, baik]","[sangat, baik]"
9542,[mantap],[mantap]
328152,"[anu, enak]","[anu, enak]"
341287,"[driver, yang, baik]","[driver, yang, baik]"


In [ ]:
#Save Dataframe into CSV
df_mxan_lower.to_csv("MaximReview_Lowercase.csv", index=False)


## PUNCTUATION REMOVAL

In [ ]:
import string

def remove_punctuation(text):
    if isinstance(text, str):
        translator = str.maketrans('', '', string.punctuation)
        cleaned_text = text.translate(translator)
        return cleaned_text
    else:
        return text


df_mxan_lower['punc_rem_content'] = df_mxan_lower['lower_tokenized_content'].apply(remove_punctuation)
df_mxan_punc_rem = df_mxan_lower[['lower_tokenized_content', 'punc_rem_content']]
df_mxan_punc_rem

,lower_tokenized_content,punc_rem_content
325768,"['murah', 'ya', '..', 'bisa', 'buat', 'langgan...",murah ya bisa buat langganan nih 👌 semoga tet...
373079,"['zte', 'sdr', 'derr', 'cut', 'pt']",zte sdr derr cut pt
13536,['oklh'],oklh
31892,"['sangat', 'baik']",sangat baik
442320,"['the', 'best']",the best
431309,['oke'],oke
86580,"['sangat', 'baik']",sangat baik
9542,['mantap'],mantap
328152,"['anu', 'enak']",anu enak
341287,"['driver', 'yang', 'baik']",driver yang baik
